## Import Packages

In [10]:
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## Import Data

In [97]:
url = 'https://homes.ksl.com/search/sw/-112.20523259300246;40.07784421059566/ne/-111.20417292445408;40.80005403788218'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)
for _ in range(80):  # Adjust the number of times based on the amount of content to load
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(4)

houses = driver.find_elements(By.CLASS_NAME, 'Listings_GridItemWrap__VnOPm') 

costs = []
addresses = []
bedrooms = []
bathrooms = []
sqft = []


for house in houses:
    # cost = house.find_element(By.XPATH, './/div[contains(@class, "GridItem_GridItemPrice__Cdf3J")]').text
    # costs.append(cost)
    # addy = house.find_element(By.XPATH, './/div[contains(@class, "GridItem_GridItemTitle__k35xf")]').text
    # addresses.append(addy)
    # details_div = house.find_element(By.XPATH, './/div[contains(@class, "GridItem_GridItemDetail__VRLk2")]')
    # spans = details_div.find_elements(By.XPATH, './/span')
    # bedroom = spans[0].text
    # bathroom = spans[1].text
    # square_feet = spans[2].text
    # bedrooms.append(bedroom)
    # bathrooms.append(bathroom)
    # sqft.append(square_feet)
    try:
        cost = house.find_element(By.XPATH, './/div[contains(@class, "GridItem_GridItemPrice__Cdf3J")]').text
        address = house.find_element(By.XPATH, './/div[contains(@class, "GridItem_GridItemTitle__k35xf")]').text
        
        details_div = house.find_element(By.XPATH, './/div[contains(@class, "GridItem_GridItemDetail__VRLk2")]')
        spans = details_div.find_elements(By.XPATH, './/span')
        bedroom_text = spans[0].text
        bathroom_text = spans[1].text
        square_feet_text = spans[2].text
    except:
        cost = "NA"
        address = "NA"
        bedroom_text = "NA"
        bathroom_text = "NA"
        square_feet_text = "NA"

    costs.append(cost)
    addresses.append(address)
    bedrooms.append(bedroom_text)
    bathrooms.append(bathroom_text)
    sqft.append(square_feet_text)

driver.quit()

## Clean Data

In [153]:
data = {
    'Price': costs,
    'Address': addresses,
    'Bedrooms': bedrooms,
    'Bathrooms': bathrooms,
    'Sqft': sqft 
}

homes = pd.DataFrame(data)

In [34]:
homes = pd.read_csv('homes.csv')

In [154]:
# Make new columns out of address column
homes['ZipCode'] = homes['Address'].str.extract(r'(\d{5}$)')
homes['State'] = homes['Address'].str.extract(r'([A-Za-z]{2})\s*\d{5}$')
homes['City'] = homes['Address'].str.extract(r'\b([^,]+)\s*,\s*[A-Za-z]{2}\s*\d{5}$')

# Clean Bedrooms column
homes['Bedrooms'] = homes['Bedrooms'].str.extract(r'(\d+\.\d+|\d+)')
homes['Bedrooms'] = pd.to_numeric(homes['Bedrooms'])

# Clean Bathrooms column
homes['Bathrooms'] = homes['Bathrooms'].str.extract(r'(\d+\.\d+|\d+)')
homes['Bathrooms'] = pd.to_numeric(homes['Bathrooms'])

# Clean Sqft column
homes['Sqft'] = homes['Sqft'].str.replace(',', '')
homes['Sqft'] = homes['Sqft'].str.extract(r'(\d+)')
homes['Sqft'] = pd.to_numeric(homes['Sqft'])

# Clean Price column
homes['Price'] = homes['Price'].replace('[\$,]', '', regex=True).str.extract(r'(\d+)')
homes['Price'] = pd.to_numeric(homes['Price'], errors='coerce')
homes['Price'] = homes['Price'].fillna(0).round().astype(int)

# Clean ZipCode column
homes['ZipCode'] = pd.to_numeric(homes['ZipCode'])

# Add in price_per_sqft
homes['Price_per_sqft'] = homes['Price'] / homes['Sqft']

# Add in lat and long
addresses = homes['Address']

latitudes = []
longitudes = []

base_url = "https://geocode.maps.co"
forward = '/search'

for address in addresses:
    params = {'q': address}
    r = requests.get(base_url + forward, params=params)
    
    if r.status_code == 200:
        data = r.json()
        if data:
            latitudes.append(data[0]['lat'])
            longitudes.append(data[0]['lon'])
        else:
            latitudes.append(None)
            longitudes.append(None)
    else:
        latitudes.append(None)
        longitudes.append(None)

    time.sleep(1)

homes['Latitude'] = latitudes
homes['Longitude'] = longitudes


# Map each city to its appropriate county
city_county_mapping = {
    'Herriman': 'Salt Lake County',
    'Springville': 'Utah County',
    'Park City': 'Summit County',
    'South Jordan': 'Salt Lake County',
    'Salt Lake City': 'Salt Lake County',
    'Sandy': 'Salt Lake County',
    'Cedar Hills': 'Utah County',
    'West Valley City': 'Salt Lake County',
    'Holladay': 'Salt Lake County',
    'Draper': 'Salt Lake County',
    'Midway': 'Wasatch County',
    'Lehi': 'Utah County',
    'Murray': 'Salt Lake County',
    'Lindon': 'Utah County',
    'Midvale': 'Salt Lake County',
    'Pleasant Grove': 'Utah County',
    'Magna': 'Salt Lake County',
    'Eagle Mountain': 'Utah County',
    'Provo': 'Utah County',
    'Orem': 'Utah County',
    'Riverton': 'Salt Lake County',
    'Spanish Fork': 'Utah County',
    'Kamas': 'Summit County',
    'West Jordan': 'Salt Lake County',
    'Alpine': 'Utah County',
    'Millcreek': 'Salt Lake County',
    'American Fork': 'Utah County',
    'Taylorsville': 'Salt Lake County',
    'Brighton': 'Salt Lake County',
    'Mapleton': 'Utah County',
    'Highland': 'Utah County',
    'Hideout': 'Wasatch County',
    'Heber City': 'Wasatch County',
    'Bluffdale': 'Salt Lake County',
    'Kearns': 'Salt Lake County',
    'Vineyard': 'Utah County',
    'Oakley': 'Summit County',
    'Copperton': 'Salt Lake County',
    'Francis': 'Summit County',
    'Cedar Valley': 'Utah County',
    'Benjamin': 'Utah County',
    'Salem': 'Utah County',
    'Saratoga Springs': 'Utah County',
    'Coalville': 'Summit County',
    'South Salt Lake': 'Salt Lake County',
    'Wanship': 'Summit County',
    'Wallsburg': 'Wasatch County',
    'W. Jordan': 'Salt Lake County',
    'Sundance': 'Utah County'
}

# Create a new 'County' column based on the mapping
homes['County'] = homes['City'].map(city_county_mapping).fillna('Unknown')

In [41]:
homes.to_csv('homes.csv', index=True)